In [2]:
import os
import cv2
import pickle
import numpy as np
from imutils import paths

In [4]:
from sklearn.preprocessing import LabelBinarizer

In [5]:
from sklearn.model_selection import train_test_split

In [8]:
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import ResNet50
from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout

from keras.models import Model
from keras.optimizers import SGD

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [5]:
datapath=r"F:\sgp_5sem\data"
outputmodel=r"F:\sgp_5sem\output\videoclassificationModel"
outputlabelbinarizer=r"F:\sgp_5sem\output\videoclassificationbinarizer"
epoch=35

In [6]:
Sports_Labels=set(['bowled','catchout','boundry','runout'])
print("images being loaded")
pathtoimages=list(paths.list_images(datapath))
data=[]
labels=[]

for images in pathtoimages:
    label=images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image=cv2.imread(images)
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(244,244))
    data.append(image)
    labels.append(label)

images being loaded


In [7]:
data=np.array(data)
labels=np.array(labels)

lb=LabelBinarizer()
labels=lb.fit_transform(labels)

In [8]:
(X_train, X_test,Y_train, Y_test)=train_test_split(data,labels,test_size=0.25, stratify=labels,random_state=42)

In [9]:
trainingAugmentation=ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

validationAugmentation=ImageDataGenerator()
mean=np.array([123.68,116.779,103.939],dtype="float32")
trainingAugmentation.mean=mean
validationAugmentation.mean=mean

In [10]:
baseModel=ResNet50(weights="imagenet",include_top=False, input_tensor=Input(shape=(244,244,3)))

headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
    headModel=Dense(5112,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(len(lb.classes_),activation="softmax")(headModel)
model=Model(inputs=baseModel.input,outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable=False

In [11]:
opt=SGD(lr=0.0001,momentum=0.9,decay=0.0001/epoch)

In [12]:
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [13]:
History=model.fit_generator(
    trainingAugmentation.flow(X_train,Y_train,batch_size=32),
    steps_per_epoch=len(X_train)//32,
    validation_data=validationAugmentation.flow(X_test,Y_test),
    validation_steps=len(X_test)//32,
    epochs=epoch
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/35
74/74 [==============================] - 262s 4s/step - loss: 1.2401 - accuracy: 0.4537 - val_loss: 0.8736 - val_accuracy: 0.6562
Epoch 2/35
74/74 [==============================] - 243s 3s/step - loss: 0.8706 - accuracy: 0.6553 - val_loss: 0.6378 - val_accuracy: 0.8047
Epoch 3/35
74/74 [==============================] - 231s 3s/step - loss: 0.7228 - accuracy: 0.7296 - val_loss: 0.5518 - val_accuracy: 0.8229
Epoch 4/35
74/74 [==============================] - 244s 3s/step - loss: 0.6470 - accuracy: 0.7564 - val_loss: 0.4854 - val_accuracy: 0.8438
Epoch 5/35
74/74 [==============================] - 239s 3s/step - loss: 0.5807 - accuracy: 0.7861 - val_loss: 0.4456 - val_accuracy: 0.8438
Epoch 6/35
74/74 [==============================] - 246s 3s/step - loss: 0.5461 - accuracy: 0.8048 - val_loss: 0.4225 - val_accuracy: 0.8607
Epoch 7/35
74/74 [==============================] - 233s 3s/step - loss: 0.523

In [15]:
model.save(outputmodel)
lbinarizer=open(r"F:\sgp_5sem\output\videoclassificationbinarizer.pickle","wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()

INFO:tensorflow:Assets written to: F:\sgp_5sem\output\videoclassificationModel\assets
